# Introduction

## Schema from Exports

Slimming Down the Columns to reduce the memory impact on the computer.

```python
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12291 entries, 0 to 12290
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              12291 non-null  int64  
 1   index                   12291 non-null  int64  
 2   Cube Name               12291 non-null  object 
 3   Cubecon Type            12291 non-null  object 
 4   cardID                  12291 non-null  object 
 5   addedTmsp               12291 non-null  object 
 6   details_name            12291 non-null  object 
 7   details_full_name       12291 non-null  object 
 8   details_artist          12291 non-null  object 
 9   details_rarity          12291 non-null  object 
 10  details_color_identity  12291 non-null  object 
 11  details_colors          12291 non-null  object 
 12  details_set             12291 non-null  object 
 13  details_released_at     12291 non-null  object 
 14  details_cmc             12291 non-null  int64  
 15  details_parsed_cost     12291 non-null  object 
 16  details_type            12291 non-null  object 
 17  details_elo             12291 non-null  float64
 18  details_popularity      12291 non-null  float64
 19  details_cubeCount       12291 non-null  int64  
 20  details_loyalty         268 non-null    object 
 21  details_power           5286 non-null   object 
 22  details_toughness       5286 non-null   object 
dtypes: float64(2), int64(4), object(17)
memory usage: 2.2+ MB
```

Count of Cards per Cubes:

```
# Make a DataFrame of above
cube_munity = cubes_main_event.groupby('details_name')['Cube Name'].nunique().sort_values(ascending=False)
cube_munity.to_csv('../Data Files/cards_per_unique_cubes.csv')
```

Creating a function to clean up the Data:

```python
# Give me the # of Cubes each card (overall card name) is in
# Then we turn it into a function:
# Then you move this to after the extraction

def tweak_cubes(cubes_main_event):
    return (cubes_main_event
     .assign(details_cmc=cubes_main_event.details_cmc.fillna(0).astype('int8'),
             is_creature=cubes_main_event.details_type.str.contains('Creature'),
             is_land = cubes_main_event.details_type.str.contains('Land'),
             is_pwer = cubes_main_event.details_type.str.contains('Planeswalker'),
             added_to_cube_on = pd.to_datetime(cubes_main_event.addedTmsp, errors='coerce', unit='ms'),
             composite_id = cubes_main_event['Cube Name'] + '-' + cubes_main_event['cardID']
        )
     .astype({'Cubecon Type': 'category', 'details_cmc': 'int8', 'details_rarity': 'category'})
     .drop(columns=['Unnamed: 0', 'index'])
)

cleaned_data = tweak_cubes(cubes_main_event)
```

Checking for Values in Added Timestamp:
```python
cleaned_data.loc[cleaned_data['added_to_cube_on'].isnull(),['added_to_cube_on', 'addedTmsp']]

cleaned_data.loc[cleaned_data['added_to_cube_on'].isnull(),'added_to_cube_on'] = pd.to_datetime(cleaned_data.loc[cleaned_data['added_to_cube_on'].isnull(),:]['addedTmsp'])
cleaned_data['added_to_cube_on'] = pd.to_datetime(cleaned_data['added_to_cube_on'], utc=True)
cleaned_data['days_until_added_to_cube'] = ((cleaned_data['added_to_cube_on'])-pd.to_datetime(cleaned_data['details_released_at']).dt.tz_convert('UTC')).dt.days

```




In [15]:
import numpy as np
import pandas as pd

In [16]:
pre_accepted_cubes = pd.read_csv('../Data Files/pre_accepted_cubes_extract.csv')
poll_winner_cubes = pd.read_csv('../Data Files/poll_winner_cubes_extract.csv')

In [17]:
pre_accepted_cubes.info()
poll_winner_cubes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12253 entries, 0 to 12252
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              12253 non-null  int64  
 1   index                   12253 non-null  int64  
 2   Cube Name               12253 non-null  object 
 3   Cubecon Type            12253 non-null  object 
 4   cardID                  12253 non-null  object 
 5   addedTmsp               12253 non-null  object 
 6   details_name            12253 non-null  object 
 7   details_full_name       12253 non-null  object 
 8   details_artist          12253 non-null  object 
 9   details_rarity          12253 non-null  object 
 10  details_color_identity  12253 non-null  object 
 11  details_colors          12253 non-null  object 
 12  details_set             12253 non-null  object 
 13  details_released_at     12253 non-null  object 
 14  details_cmc             12253 non-null

In [18]:
cubes_main_event = pd.concat([pre_accepted_cubes, poll_winner_cubes], ignore_index=True)
cubes_main_event.head()

,Unnamed: 0,index,Cube Name,Cubecon Type,cardID,addedTmsp,details_name,details_full_name,details_artist,details_rarity,...,details_released_at,details_cmc,details_parsed_cost,details_type,details_elo,details_popularity,details_cubeCount,details_loyalty,details_power,details_toughness
0,0,0,Regular Cube,Pre-Accepted,251015ed-9408-4941-894a-158551ed2613,1572901810806,Favored Hoplite,Favored Hoplite [ths-13],Winona Nelson,uncommon,...,2013-09-27,1,['w'],Creature — Human Soldier,1242.0,1.615249,2321,NaN,1,2
1,1,1,Regular Cube,Pre-Accepted,70e3a90c-1e5c-4646-b3d5-ff46d3fa7b35,1572901810808,Trusted Pegasus,Trusted Pegasus [m20-314],Chris Rahn,common,...,2019-07-12,3,"['w', '2']",Creature — Pegasus,1175.4,1.531042,2200,NaN,2,2
2,2,2,Regular Cube,Pre-Accepted,27394079-924a-4fdb-8be2-f853193eca80,1572901810808,Whitemane Lion,Whitemane Lion [a25-39],Zoltan Boros & Gabor Szikszai,common,...,2018-03-16,2,"['w', '1']",Creature — Cat,1197.6,3.714168,5337,NaN,2,2
3,3,3,Regular Cube,Pre-Accepted,b958bcdd-d0ea-4ae0-9dd0-e6de5cf74128,1572901810809,Restoration Angel,Restoration Angel [mm3-20],Johannes Voss,rare,...,2017-03-17,4,"['w', '3']",Creature — Angel,1318.2,17.904839,25728,NaN,3,4
4,4,4,Regular Cube,Pre-Accepted,c47ba1fa-3ace-488b-97e6-d9f3b389c602,1572901810809,Emeria Angel,Emeria Angel [ima-20],Jim Murray,rare,...,2017-11-17,4,"['w', 'w', '2']",Creature — Angel,1244.8,4.291789,6167,NaN,3,3


In [5]:
# Update Date when Ran
cubes_main_event.to_csv('../Data Files/cubecon_card_list_2023_08_27.csv')

In [6]:
# Give me the # of Cubes each card (overall card name) is in
# Then we turn it into a function:
# Then you move this to after the extraction

def tweak_cubes(cubes_main_event):
    return (cubes_main_event
     .assign(details_cmc=cubes_main_event.details_cmc.fillna(0).astype('int8'),
             is_creature=cubes_main_event.details_type.str.contains('Creature'),
             is_land = cubes_main_event.details_type.str.contains('Land'),
             is_planeswalker = cubes_main_event.details_type.str.contains('Planeswalker'),
             is_gold = cubes_main_event.details_colors.str.contains(','),
             added_to_cube_on = pd.to_datetime(cubes_main_event.addedTmsp, errors='coerce', unit='ms'),
             composite_id = cubes_main_event['Cube Name'] + '-' + cubes_main_event['cardID']
        )
     .astype(
         {'Cubecon Type': 'category', 
          'details_cmc': 'int8',
          'details_rarity': 'category', 
          'details_released_at':'datetime64[ns, UTC]'}
     )
     .drop(columns=['Unnamed: 0', 'index'])
)

cleaned_data = tweak_cubes(cubes_main_event)

In [7]:
cleaned_data.loc[cleaned_data['added_to_cube_on'].isnull(),'added_to_cube_on'] = pd.to_datetime(cleaned_data.loc[cleaned_data['added_to_cube_on'].isnull(),:]['addedTmsp'])
cleaned_data['added_to_cube_on'] = pd.to_datetime(cleaned_data['added_to_cube_on'], utc=True)
cleaned_data['days_until_added_to_cube'] = ((cleaned_data['added_to_cube_on'])-pd.to_datetime(cleaned_data['details_released_at']).dt.tz_convert('UTC')).dt.days


In [8]:
cleaned_data.info()
cubes_main_event.to_csv('../Data Files/cubecon_card_list_2023_07_27.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20590 entries, 0 to 20589
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   Cube Name                 20590 non-null  object             
 1   Cubecon Type              20590 non-null  category           
 2   cardID                    20590 non-null  object             
 3   addedTmsp                 20590 non-null  object             
 4   details_name              20590 non-null  object             
 5   details_full_name         20590 non-null  object             
 6   details_artist            20590 non-null  object             
 7   details_rarity            20590 non-null  category           
 8   details_color_identity    20590 non-null  object             
 9   details_colors            20590 non-null  object             
 10  details_set               20590 non-null  object             
 11  details_release

In [9]:
avg_cubing = (cleaned_data
                 .groupby('Cube Name')
                 .mean()
                 .sort_values('details_popularity', ascending=False)
                 .loc[:,['details_popularity', 'is_land', 'is_gold']]
)

avg_cubing

,details_popularity,is_land,is_gold
Cube Name,,,
Data Generated Vintage Cube,11.210681,0.150000,0.083333
The Museum of Modern,10.003723,0.235417,0.212500
The Bun Magic Cube,9.832680,0.222222,0.097222
Dekkaru Cube,9.176258,0.151852,0.085185
The Modern Darlings Cube,8.740337,0.208333,0.116667
Eleusis,8.511453,0.146667,0.057778
The Chicago Cube,7.912523,0.141667,0.105556
Derek’s Cube,7.773991,0.208333,0.172917
The Creative Cube,7.521774,0.200000,0.090000


In [10]:
cleaned_data.groupby('details_name')['Cube Name'].nunique().sort_values(ascending=False).head(40)

details_name
Temple Garden              31
Stomping Ground            31
Overgrown Tomb             30
Sacred Foundry             30
Blood Crypt                30
Godless Shrine             29
Watery Grave               29
Breeding Pool              29
Hallowed Fountain          29
Steam Vents                29
Lightning Bolt             27
Windswept Heath            26
Bloodstained Mire          26
Wooded Foothills           26
Faithless Looting          26
Verdant Catacombs          25
Flooded Strand             25
Path to Exile              25
Polluted Delta             25
Marsh Flats                24
Misty Rainforest           24
Scalding Tarn              24
Arid Mesa                  24
Eternal Witness            23
Thraben Inspector          23
Young Pyromancer           23
Duress                     23
Abrade                     23
Tireless Tracker           22
Carrion Feeder             22
Grim Lavamancer            21
Mana Leak                  20
Unearth                    

In [55]:
peasant = cleaned_data[cleaned_data['details_rarity'].isin(['uncommon', 'common'])]

In [56]:
peasant_groupby = peasant.groupby('details_name')['Cube Name'].nunique().sort_values(ascending=False)
peasant_groupby.to_csv('../Data Files/peasant_cards_grouping_2023_06_22.csv')

In [11]:
cleaned_data[cleaned_data['details_set'] == 'ltr'].groupby('details_name')['Cube Name'].nunique().sort_values(ascending=False).head(40)

details_name
Andúril, Flame of the West      7
Reprieve                        7
Delighted Halfling              5
Stern Scolding                  5
Rosie Cotton of South Lane      5
Samwise Gamgee                  5
Merry, Esquire of Rohan         5
Wizard's Rockets                5
The One Ring                    4
Palantír of Orthanc             4
The Shire                       4
Pippin, Guard of the Citadel    4
Rally at the Hornburg           4
Flowering of the White Tree     4
Rise of the Witch-king          4
Orcish Bowmasters               3
Éomer, Marshal of Rohan         3
Improvised Club                 3
Gollum, Patient Plotter         3
Gollum's Bite                   3
Lembas                          3
Minas Tirith                    3
Gandalf the White               3
Flame of Anor                   3
Denethor, Ruling Steward        3
Barrow-Blade                    3
Arwen, Mortal Queen             3
Mirkwood Bats                   2
Generous Ent                    2
G

In [12]:
cleaned_data[cleaned_data['details_set'] == 'ltr'].groupby('Cube Name')['details_name'].nunique().sort_values(ascending=False)

Cube Name
Amonkar Desert                      23
The Spikeless Cube                  19
Derek’s Cube                        19
The Live the Dream Cube             14
Creatureless Cube                   12
The Creative Cube                   12
The Tempo Cube                      11
Commander extravaganza!             10
Tiny Leaders                         9
The Jund Cube                        9
A Study in Harmony                   8
The Buildaround Cube                 6
The Devoid Cube                      6
The Penrose Cube                     6
Sammich's Peasant Cube               5
The Bun Magic Cube                   5
All-Foil Midrange Cube               4
Changeling Cube                      4
Loial's Micro Cube                   4
Emma Partlow's Peasant Cube          4
Counters of Monte Cristo             3
Vehicle Cube: Eiganjo Drift          2
Spooky Black Halloween Graveyard     2
Uber Bear's Artifact Cube            2
The Boat Cube                        2
Name: details_n

In [13]:
cleaned_data[cleaned_data['details_name'] == 'Karakas'].groupby('Cube Name')['details_name'].nunique().sort_values(ascending=False)

Cube Name
Tolsimir Cube                  1
Data Generated Vintage Cube    1
Name: details_name, dtype: int64

In [14]:
cleaned_data[cleaned_data['details_set'] == 'ltr'].groupby('Cube Name')['details_name'].nunique().sort_values(ascending=False)

Cube Name
Amonkar Desert                      23
The Spikeless Cube                  19
Derek’s Cube                        19
The Live the Dream Cube             14
Creatureless Cube                   12
The Creative Cube                   12
The Tempo Cube                      11
Commander extravaganza!             10
Tiny Leaders                         9
The Jund Cube                        9
A Study in Harmony                   8
The Buildaround Cube                 6
The Devoid Cube                      6
The Penrose Cube                     6
Sammich's Peasant Cube               5
The Bun Magic Cube                   5
All-Foil Midrange Cube               4
Changeling Cube                      4
Loial's Micro Cube                   4
Emma Partlow's Peasant Cube          4
Counters of Monte Cristo             3
Vehicle Cube: Eiganjo Drift          2
Spooky Black Halloween Graveyard     2
Uber Bear's Artifact Cube            2
The Boat Cube                        2
Name: details_n